In [2]:
import os
os.getpid()#看看有没有在用gpu

69010

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [5]:
import pandas as pd
import numpy  as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

import scipy 
import joblib
from netCDF4 import Dataset
import netCDF4 as nc 
from tensorflow.keras.callbacks import LearningRateScheduler, Callback
import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import Input 



/home/user1/.conda/envs/ENIO/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/user1/.conda/envs/ENIO/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/user1/.conda/envs/ENIO/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/user1/.conda/envs/ENIO/lib/python3.6/site-packages/tensorf

In [6]:
soda_label = pd.read_csv('./ENSO/data/df_SODA_label.csv')['label']
sst_SODA = pd.read_csv('./ENSO/data/df_sst_SODA.csv')
t300_SODA = pd.read_csv('./ENSO/data/df_t300_SODA.csv')
ua_SODA = pd.read_csv('./ENSO/data/df_ua_SODA.csv')
va_SODA = pd.read_csv('./ENSO/data/df_va_SODA.csv')
sst_SODA.pop('year_month')
t300_SODA.pop('year_month')
va_SODA.pop('year_month')
ua_SODA.pop('year_month')
sst_SODA = np.array(sst_SODA).reshape(100,36,24,72,1)[:,:12,:,:,:]
t300_SODA = np.array(t300_SODA).reshape(100,36,24,72,1)[:,:12,:,:,:]
va_SODA = np.array(va_SODA).reshape(100,36,24,72,1)[:,:12,:,:,:]
ua_SODA = np.array(ua_SODA).reshape(100,36,24,72,1)[:,:12,:,:,:]



In [7]:
### 训练特征，保证和训练集一致
tr_features = np.concatenate([sst_SODA,t300_SODA,va_SODA,ua_SODA],axis=-1)
tr_features.shape

(100, 12, 24, 72, 4)

In [8]:
### 训练标签，取后24个
tr_labels = np.array(soda_label).reshape(-1,36)[:,12:]
tr_labels.shape

(100, 24)

In [9]:

### 训练集验证集划分
tr_len     = int(tr_features.shape[0] * 0.8)
tr_fea     = tr_features[:tr_len,:].copy()
tr_label   = tr_labels[:tr_len,:].copy()
 
val_fea     = tr_features[tr_len:,:].copy()
val_label   = tr_labels[tr_len:,:].copy()

In [10]:
def RMSE(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

def RMSE_fn(y_true, y_pred):
    return np.sqrt(np.mean(np.power(np.array(y_true, float).reshape(-1, 1) - np.array(y_pred, float).reshape(-1, 1), 2)))

def build_model():
    input_shape = (12,24,72,4)
    inp    = Input(shape=input_shape) 
    x    = Conv3D(64,(4,16,41), activation='relu', input_shape=input_shape)(inp)
    x = MaxPooling3D((2,2,2))(x)
    x    = Conv3D(128,(4,4,16), activation='relu')(x)
    # x = MaxPooling3D((4,4,1))(x)
    # x    = Conv3D(128,2, activation='relu')(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)  
    x = Dropout(0.25)(x) 
    x = Dense(32, activation='relu')(x)   
    x = Dropout(0.25)(x)  
    output = Dense(24, activation='linear')(x)   

    model  = Model(inputs=inp, outputs=output)
    adam = tf.keras.optimizers.Adam(lr=1e-3,beta_1=0.99,beta_2 = 0.99) 
    model.compile(optimizer=adam, loss=RMSE)
    return model
#### 构建模型
model_cnn     = build_model()
#### 模型存储的位置
model_weights = './model/model_mlp_baseline.h5'
#保存最佳模型
checkpoint = ModelCheckpoint(model_weights, monitor='val_loss', verbose=0, save_best_only=True, mode='min',
                             save_weights_only=True)
#Reduce learning rate when a metric has stopped improving.
plateau        = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, min_delta=1e-4, mode='min')
early_stopping = EarlyStopping(monitor="val_loss", patience=20)
model_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 12, 24, 72, 4)     0         
_________________________________________________________________
conv3d (Conv3D)              (None, 9, 9, 32, 64)      671808    
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 4, 4, 16, 64)      0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 1, 1, 1, 128)      2097280   
_________________________________________________________________
flatten (Flatten)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
__________

In [ ]:
history        = model_cnn.fit(tr_fea,tr_label,
                    validation_data=(val_fea, val_label),
                    batch_size=4096, epochs=2000,
                    callbacks=[plateau, checkpoint, early_stopping],
                    verbose=1)

Train on 80 samples, validate on 20 samples
Epoch 1/2000
80/80 [==============================] - 219s 3s/step - loss: 0.8309 - val_loss: 1.2677
Epoch 2/2000
80/80 [==============================] - 216s 3s/step - loss: 1.6511 - val_loss: 1.7875
Epoch 3/2000
80/80 [==============================] - 216s 3s/step - loss: 1.6613 - val_loss: 1.3791
Epoch 4/2000
80/80 [==============================] - 217s 3s/step - loss: 1.1585 - val_loss: 1.0380
Epoch 5/2000
80/80 [==============================] - 215s 3s/step - loss: 0.9852 - val_loss: 1.1403
Epoch 6/2000
80/80 [==============================] - 214s 3s/step - loss: 0.9827 - val_loss: 1.3063
Epoch 7/2000
80/80 [==============================] - 210s 3s/step - loss: 1.0553 - val_loss: 1.2360
Epoch 8/2000
80/80 [==============================] - 214s 3s/step - loss: 1.0209 - val_loss: 1.1691
Epoch 9/2000
80/80 [==============================] - 217s 3s/step - loss: 0.9535 - val_loss: 0.9645
Epoch 10/2000


In [ ]:
del val_label
del val_fea
del tr_fea
del tr_label
del sst_SODA
del t300_SODA
del ua_SODA
del va_SODA
import gc
gc.collect()

In [ ]:

sst_CMIP = pd.read_csv('./ENSO/data/df_sst_CMIP.csv')
t300_CMIP = pd.read_csv('./ENSO/data/df_t300_CMIP.csv')
ua_CMIP = pd.read_csv('./ENSO/data/df_ua_CMIP.csv')
va_CMIP = pd.read_csv('./ENSO/data/df_va_CMIP.csv')


cmip_label = pd.read_csv('./ENSO/data/df_CMIP_label.csv')['label']


sst_CMIP.pop('year_month')
t300_CMIP.pop('year_month')
va_CMIP.pop('year_month')
ua_CMIP.pop('year_month')
sst_CMIP = np.array(sst_CMIP).reshape(-1,36,24,72,1)[:,:12,:,:,:]
t300_CMIP = np.array(t300_CMIP).reshape(-1,36,24,72,1)[:,:12,:,:,:]
va_CMIP = np.array(va_CMIP).reshape(-1,36,24,72,1)[:,:12,:,:,:]
ua_CMIP = np.array(ua_CMIP).reshape(-1,36,24,72,1)[:,:12,:,:,:]
tr_features2 = np.concatenate([sst_CMIP,t300_CMIP,va_CMIP,ua_CMIP],axis=-1)
tr_features.shape
tr_labels2 = np.array(cmip_label).reshape(-1,36)[:,12:]
tr_labels.shape

### 训练集验证集划分
tr_len2     = int(tr_features.shape[0] * 0.8)
tr_fea2     = tr_features[:tr_len,:].copy()
tr_label2   = tr_labels[:tr_len,:].copy()
 
val_fea2     = tr_features[tr_len:,:].copy()
val_label2   = tr_labels[tr_len:,:].copy()


history        = model_cnn.fit(tr_fea2,tr_label2,
                    validation_data=(val_fea2, val_label2),
                    batch_size=4096, epochs=2000,
                    callbacks=[plateau, checkpoint, early_stopping],
                    verbose=1)

In [ ]:
model_cnn = build_model()
model_cnn.load_weights('./model/model_mlp_baseline.h5')

prediction = model_cnn.predict(val_fea)

In [ ]:

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.power(np.array(y_true, float).reshape(-1, 1) - np.array(y_pred, float).reshape(-1, 1), 2)))

def score(y_true, y_preds):
    accskill_score = 0
    rmse_scores    = 0
    a = [1.5] * 4 + [2] * 7 + [3] * 7 + [4] * 6
    y_true_mean = np.mean(y_true,axis=0) 
    y_pred_mean = np.mean(y_preds,axis=0) 
#     print(y_true_mean.shape, y_pred_mean.shape)

    for i in range(24): 
        fenzi = np.sum((y_true[:,i] -  y_true_mean[i]) *(y_preds[:,i] -  y_pred_mean[i]) ) 
        fenmu = np.sqrt(np.sum((y_true[:,i] -  y_true_mean[i])**2) * np.sum((y_preds[:,i] -  y_pred_mean[i])**2) ) 
        cor_i = fenzi / fenmu
    
        accskill_score += a[i] * np.log(i+1) * cor_i
        rmse_score   = rmse(y_true[:,i], y_preds[:,i])
#         print(cor_i,  2 / 3.0 * a[i] * np.log(i+1) * cor_i - rmse_score)
        rmse_scores += rmse_score 
    
    return 2 / 3.0 * accskill_score - rmse_scores


print('score', score(y_true = val_label, y_preds = prediction))